<h1>6장 프롬프트 엔지니어링</h1>
<i>프롬프트 엔지니어링으로 출력 품질을 향상시키는 방법</i>

<a href="https://github.com/rickiepark/handson-llm"><img src="https://img.shields.io/badge/GitHub%20Repository-black?logo=github"></a>
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rickiepark/handson-llm/blob/main/chapter06.ipynb)

---

이 노트북은 <[핸즈온 LLM](https://tensorflow.blog/handson-llm/)> 책 6장의 코드를 담고 있습니다.

---

<a href="https://tensorflow.blog/handson-llm/">
<img src="https://tensorflow.blog/wp-content/uploads/2025/05/ed95b8eca688ec98a8_llm.jpg" width="350"/></a>

### [선택사항] - <img src="https://colab.google/static/images/icons/colab.png" width=100>에서 패키지 선택하기


이 노트북을 구글 코랩에서 실행한다면 다음 코드 셀을 실행하여 이 노트북에서 필요한 패키지를  설치하세요.

---

💡 **NOTE**: 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.

---

LLMS의 출력 품질을 높이기 위한 전략들을 실제 코드와 함께 실습

1. 프롬프트 구조화(persona, instruction, context 등)

2. ChatML 템플릿 적용

3. sampling 기법으로 창의성·안정성 제어

4. Zero-shot / One-shot / Few-shot 비교

5. Chain-of-Thought(CoT)

6. Zero-shot CoT / Tree-of-Thought(ToT)

7. Prompt Chaining

8. JSON 스키마 강제 출력

9. Transformers vs llama.cpp 기반 실행 방식 비교

In [1]:
%%capture
# 아래 명령은 'llama-cpp-python'이라는 패키지를 설치하는 코드입니다.
# 이 패키지는 로컬 LLaMA 모델(gguf 등)을 실행할 때 필요합니다.

# 현재 코랩(Colab)은 파이썬 3.11, CUDA 12.4 환경이므로
# 해당 환경에 맞게 만들어진 휠(.whl) 파일을 직접 설치합니다.

# pip install 뒤에 있는 주소는 패키지 파일이 저장된 다운로드 링크입니다.
# 인터넷에서 파일을 가져와 설치하는 방식입니다.
!pip install https://github.com/abetlen/llama-cpp-python/releases/download/v0.3.4-cu124/llama_cpp_python-0.3.4-cp311-cp311-linux_x86_64.whl

In [2]:
# Phi-3 모델을 사용할 때, 특정 버전의 transformers 라이브러리와
# 호환성 문제가 발생할 수 있습니다.

# Phi-3는 transformers 최신 버전과 충돌하는 경우가 있기 때문에
# 안정적으로 작동하는 버전인 4.48.3을 지정하여 설치합니다.

# 아래 명령은 transformers 라이브러리를 정확히 4.48.3 버전으로 설치합니다.
!pip install transformers==4.48.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 83.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1


In [3]:
# 깃허브나 주피터 노트북 환경에서는 tqdm 진행 표시줄 때문에
# 위젯 오류(Loading widget failed 등)가 발생하는 경우가 많습니다.
# 이를 방지하기 위해 tqdm 기능을 모두 비활성화합니다.

import os
import tqdm
from transformers.utils import logging

# tqdm 기본 함수(진행 표시줄 생성 함수)를 "아무것도 하지 않는 함수"로 바꿉니다.
# iter([])는 빈 반복자를 의미 → 즉, 아무 결과도 출력하지 않음.
tqdm.tqdm = lambda *args, **kwargs: iter([])
tqdm.auto.tqdm = lambda *args, **kwargs: iter([])
tqdm.notebook.tqdm = lambda *args, **kwargs: iter([])

# 환경 변수로도 tqdm 비활성화 설정을 전달
os.environ["DISABLE_TQDM"] = "1"

# transformers 라이브러리 내부에서 사용하는 tqdm도 비활성화
logging.disable_progress_bar()

## 모델 로드하기

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# --------------------------------------------------------------
# 1. Phi-3 모델과 토크나이저 불러오기 (약 2분 소요)
# --------------------------------------------------------------
# from_pretrained():
#   - Hugging Face Hub에 있는 사전학습 모델을 내려받아 메모리에 로드합니다.
#   - "microsoft/Phi-3-mini-4k-instruct" 모델을 사용합니다.

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",

    # device_map="cuda"
    #   → 모델을 GPU(cuda)에 자동으로 배치하여 빠르게 실행되도록 함.
    device_map="cuda",

    # torch_dtype="auto"
    #   → GPU·모델 환경을 보고 자동으로 가장 적절한 데이터 타입(float16 등)을 선택.
    torch_dtype="auto",

    # trust_remote_code=True
    #   → 모델이 사용자 정의 코드(custom modeling files)를 포함할 때 로딩 허용.
    #     (Phi-3는 커스텀 코드가 있어 이 옵션이 필요)
    trust_remote_code=True,
)

# 토크나이저 불러오기
# - 텍스트를 모델이 읽을 수 있는 토큰 ID로 변환하는 처리기.
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

# --------------------------------------------------------------
# 2. 텍스트 생성 파이프라인 구성
# --------------------------------------------------------------
# pipeline():
#   - 모델 + 토크나이저 + 설정값을 묶어서 간단하게 텍스트 생성 가능.
#   - "text-generation"은 이어서 문장을 생성하는 작업.
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,

    # 입력 텍스트는 숨기고, 모델이 생성한 새로운 텍스트만 출력하도록 설정
    return_full_text=False,

    # 생성할 최대 토큰 수(단어 조각). 500개면 충분히 많은 길이의 문장 생성 가능.
    max_new_tokens=500,

    # do_sample=False
    #   → 랜덤성 없이 “가장 확률이 높은 다음 단어”만 선택.
    #     결과가 안정적이고 일관된 응답을 얻을 수 있음.
    do_sample=False,
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added mal

In [5]:
# --------------------------------------------------------------
# 3. 모델에게 전달할 입력 프롬프트 정의
# --------------------------------------------------------------
# Phi-3는 ChatGPT와 같은 '대화형' 포맷을 지원하므로
# messages라는 리스트 안에 role과 content를 넣어 전달합니다.
messages = [
    {"role": "user", "content": "Create a funny joke about chickens."}
]

# --------------------------------------------------------------
# 4. 텍스트 생성 실행
# --------------------------------------------------------------
# pipeline에 messages를 넣으면 모델이 이어지는 텍스트를 생성합니다.
output = pipe(messages)

# 생성된 텍스트만 꺼내서 출력
print(output[0]["generated_text"])

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


 Why did the chicken join the band? Because it had the drumsticks!


✔︎ apply_chat_template()가 왜 중요한가?

- Chat 모델(Phi-3, LLaMA-3 등)은 대화 규칙(ChatML, Llama-Chat format 등)을 정확하게 따라야
“role=user/assistant” 구조가 올바르게 해석됩니다.

In [6]:
# --------------------------------------------------------------
# 5. 프롬프트 템플릿(ChatML 형식) 적용
# --------------------------------------------------------------
# Phi-3와 같은 대화형 모델은 내부적으로 "특정 형식(Chat Template)"
# 에 맞춰야 정확하게 작동합니다.
#
# apply_chat_template():
#   - messages(대화 리스트)를 모델이 이해하는 문자열 형태로 변환
#   - tokenize=False → 아직 토큰 ID로 바꾸지 않고 순수 텍스트만 확인
#
# 즉, 사람이 읽기 쉬운 메시지 목록 → 모델이 읽는 대화 형식(prompt)
# 으로 변환하는 과정입니다.
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False)

# 변환된 실제 프롬프트가 어떻게 생겼는지 출력해 확인합니다.
print(prompt)

<|user|>
Create a funny joke about chickens.<|end|>
<|endoftext|>


| temperature | 특징                         |
| ----------- | -------------------------- |
| 0.0 ~ 0.3   | 매우 안정적, 정확한 답변 위주          |
| 0.7 ~ 1.0   | 적당히 창의적, 자연스러운 글 생성         |
| 1.0 ~ 1.5   | 매우 창의적, 농담·스토리·아이디어 생성에 유리 |
| > 1.5       | 의미 일탈·헛소리 발생 가능            |


In [10]:
# --------------------------------------------------------------
# 6. 더 창의적인 출력 생성하기 (temperature 조절)
# --------------------------------------------------------------
# do_sample=True :
#   - 모델이 "가장 확률이 높은 단어만" 고르지 않고,
#     여러 후보 중에서 무작위성을 적용해 선택하도록 합니다.
#
# temperature=1 :
#   - 무작위성(창의성)의 강도를 결정하는 값.
#   - 값이 높을수록 (예: 1.0 ~ 1.5) 더 다양한 표현을 사용하고,
#     값이 낮을수록 (예: 0.1 ~ 0.5) 더 보수적이고 논리적인 문장을 생성합니다.
#
# 여기서는 temperature=1을 사용해 “창의적이고 재밌는” 응답을 생성하려는 설정입니다.
output = pipe(messages, do_sample=True, temperature=1)

# 생성된 텍스트 출력
print(output[0]["generated_text"])

 Why did the chicken join the band?

Because it had the drumsticks!


| top_p 값   | 특징                  |
| --------- | ------------------- |
| 0.5       | 매우 제한된 단어만 사용 → 안정적 |
| 0.8 ~ 0.9 | 적당한 다양성, 자연스러운 문장   |
| 1.0       | 제약 없음, 창의성 최대로 증가   |


In [11]:
# --------------------------------------------------------------
# 7. top_p 설정하기 (Nucleus Sampling)
# --------------------------------------------------------------
# do_sample=True :
#   - 무작위 샘플링을 활성화합니다.
#
# top_p :
#   - "누적 확률"을 기준으로 단어 후보를 선택하는 방법입니다.
#   - 예: top_p=0.9 → 전체 확률이 위에서부터 90%가 되는 단어들만 후보로 사용.
#
# top_p=1 :
#   - 제한 없이 모든 단어 후보를 사용할 수 있음.
#   - 즉, 창의적인 단어가 나올 가능성이 더 높아짐.
#
# temperature가 '얼마나 무작위적으로 선택할지'를 조절한다면,
# top_p는 '어떤 후보 범위를 선택지로 볼 것인지'를 조절하는 개념입니다.
#
output = pipe(messages, do_sample=True, top_p=1)

# 생성된 텍스트 출력
print(output[0]["generated_text"])

 Why do chickens make terrible magicians?

They always peck at their tricks!


## 고급 프롬프트 엔지니어링


### 복잡한 프롬프트

| 프롬프트 요소     | 효과                     |
| ----------- | ---------------------- |
| persona     | 모델이 “전문가”처럼 사고하도록 유도   |
| instruction | 요약해야 한다는 작업 지시         |
| context     | 어떤 관점/기준으로 요약해야 하는지 강조 |
| data_format | 출력 형태를 일관되게 유지         |
| audience    | 글의 난이도·톤을 조절           |
| tone        | 전문적이고 명확한 스타일로 제한      |
| data        | 요약 대상 텍스트 제공           |


In [12]:
# --------------------------------------------------------------
# 긴 연구 논문 텍스트를 요약하기 위한 프롬프트 구성 예제
# --------------------------------------------------------------

# 원문 텍스트 (요약할 내용)
# → 실제 요약하려는 텍스트로 바꿔 넣으면 됩니다.

# https://jalammar.github.io/illustrated-transformer/에서 가져온 텍스트
text = """In the previous post, we looked at Attention – a ubiquitous method in modern deep learning models. Attention is a concept that helped improve the performance of neural machine translation applications. In this post, we will look at The Transformer – a model that uses attention to boost the speed with which these models can be trained. The Transformer outperforms the Google Neural Machine Translation model in specific tasks. The biggest benefit, however, comes from how The Transformer lends itself to parallelization. It is in fact Google Cloud’s recommendation to use The Transformer as a reference model to use their Cloud TPU offering. So let’s try to break the model apart and look at how it functions.
The Transformer was proposed in the paper Attention is All You Need. A TensorFlow implementation of it is available as a part of the Tensor2Tensor package. Harvard’s NLP group created a guide annotating the paper with PyTorch implementation. In this post, we will attempt to oversimplify things a bit and introduce the concepts one by one to hopefully make it easier to understand to people without in-depth knowledge of the subject matter.
Let’s begin by looking at the model as a single black box. In a machine translation application, it would take a sentence in one language, and output its translation in another.
Popping open that Optimus Prime goodness, we see an encoding component, a decoding component, and connections between them.
The encoding component is a stack of encoders (the paper stacks six of them on top of each other – there’s nothing magical about the number six, one can definitely experiment with other arrangements). The decoding component is a stack of decoders of the same number.
The encoders are all identical in structure (yet they do not share weights). Each one is broken down into two sub-layers:
The encoder’s inputs first flow through a self-attention layer – a layer that helps the encoder look at other words in the input sentence as it encodes a specific word. We’ll look closer at self-attention later in the post.
The outputs of the self-attention layer are fed to a feed-forward neural network. The exact same feed-forward network is independently applied to each position.
The decoder has both those layers, but between them is an attention layer that helps the decoder focus on relevant parts of the input sentence (similar what attention does in seq2seq models).
Now that we’ve seen the major components of the model, let’s start to look at the various vectors/tensors and how they flow between these components to turn the input of a trained model into an output.
As is the case in NLP applications in general, we begin by turning each input word into a vector using an embedding algorithm.
Each word is embedded into a vector of size 512. We'll represent those vectors with these simple boxes.
The embedding only happens in the bottom-most encoder. The abstraction that is common to all the encoders is that they receive a list of vectors each of the size 512 – In the bottom encoder that would be the word embeddings, but in other encoders, it would be the output of the encoder that’s directly below. The size of this list is hyperparameter we can set – basically it would be the length of the longest sentence in our training dataset.
After embedding the words in our input sequence, each of them flows through each of the two layers of the encoder.
Here we begin to see one key property of the Transformer, which is that the word in each position flows through its own path in the encoder. There are dependencies between these paths in the self-attention layer. The feed-forward layer does not have those dependencies, however, and thus the various paths can be executed in parallel while flowing through the feed-forward layer.
Next, we’ll switch up the example to a shorter sentence and we’ll look at what happens in each sub-layer of the encoder.
Now We’re Encoding!
As we’ve mentioned already, an encoder receives a list of vectors as input. It processes this list by passing these vectors into a ‘self-attention’ layer, then into a feed-forward neural network, then sends out the output upwards to the next encoder.
"""

# --------------------------------------------------------------
# 1. 프롬프트 구성 요소 (Prompt Engineering)
# --------------------------------------------------------------
# persona :
#   모델에게 "어떤 역할"을 맡길지 지정합니다.
#   → 전문가는 전문적인 요약을, 교사는 쉬운 설명을 제공하는 등
persona = (
    "You are an expert in Large Language models. "
    "You excel at breaking down complex papers into digestible summaries.\n"
)

# instruction :
#   모델에게 "무엇을 하라"고 지시하는 문장.
instruction = "Summarize the key findings of the paper provided.\n"

# context :
#   작업을 수행할 때 어떤 관점을 유지해야 하는지 제공.
context = (
    "Your summary should extract the most crucial points that can help "
    "researchers quickly understand the most vital information of the paper.\n"
)

# data_format :
#   출력 결과를 어떤 형태로 내보낼지 지정 (bullet point, list, paragraph 등)
data_format = (
    "Create a bullet-point summary that outlines the method. "
    "Follow this up with a concise paragraph that encapsulates the main results.\n"
)

# audience :
#   글의 대상 독자 → 문체와 깊이 조절에 도움이 됨.
audience = (
    "The summary is designed for busy researchers that quickly need to "
    "grasp the newest trends in Large Language Models.\n"
)

# tone :
#   문체(톤) 설정 → 전문적, 친절, 캐주얼 등 선택 가능.
tone = "The tone should be professional and clear.\n"

# data :
#   실제로 요약할 텍스트를 포함하는 부분
data = f"Text to summarize: {text}"

# --------------------------------------------------------------
# 2. 최종 프롬프트 생성
# --------------------------------------------------------------
# 요소들을 하나로 합쳐 모델에 전달할 "query" 프롬프트를 만듭니다.
query = persona + instruction + context + data_format + audience + tone + data

# 생성된 프롬프트가 어떤 형태인지 확인
print(query)

You are an expert in Large Language models. You excel at breaking down complex papers into digestible summaries.
Summarize the key findings of the paper provided.
Your summary should extract the most crucial points that can help researchers quickly understand the most vital information of the paper.
Create a bullet-point summary that outlines the method. Follow this up with a concise paragraph that encapsulates the main results.
The summary is designed for busy researchers that quickly need to grasp the newest trends in Large Language Models.
The tone should be professional and clear.
Text to summarize: In the previous post, we looked at Attention – a ubiquitous method in modern deep learning models. Attention is a concept that helped improve the performance of neural machine translation applications. In this post, we will look at The Transformer – a model that uses attention to boost the speed with which these models can be trained. The Transformer outperforms the Google Neural Machin

Phi-3 모델은 단순한 텍스트 입력을 그대로 쓰지 않고
특정한 대화 형식(ChatML 포맷)을 따라야 합니다.

In [13]:
# --------------------------------------------------------------
# 3. 프롬프트를 Phi-3(ChatML) 형식으로 자동 변환
# --------------------------------------------------------------
# messages:
#   - Chat 모델이 요구하는 입력 형식: role + content 구조
#   - 여기서는 전체 query(프롬프트)를 user 역할로 전달합니다.
messages = [
    {"role": "user", "content": query}
]

# Chat 템플릿 적용:
#   - tokenizer.apply_chat_template():
#       messages 구조를 Phi-3 모델이 읽는 내부 포맷(ChatML)으로 자동 변환
#       (예: <|user|> ... <|assistant|> 형태)
#
#   - tokenize=False:
#       토큰 ID로 변환하지 않고, 단순 문자열로 확인만 하는 옵션
#       디버깅에 매우 유용 → 모델이 실제로 어떤 프롬프트를 입력받는지 눈으로 볼 수 있음
print(tokenizer.apply_chat_template(messages, tokenize=False))

<|user|>
You are an expert in Large Language models. You excel at breaking down complex papers into digestible summaries.
Summarize the key findings of the paper provided.
Your summary should extract the most crucial points that can help researchers quickly understand the most vital information of the paper.
Create a bullet-point summary that outlines the method. Follow this up with a concise paragraph that encapsulates the main results.
The summary is designed for busy researchers that quickly need to grasp the newest trends in Large Language Models.
The tone should be professional and clear.
Text to summarize: In the previous post, we looked at Attention – a ubiquitous method in modern deep learning models. Attention is a concept that helped improve the performance of neural machine translation applications. In this post, we will look at The Transformer – a model that uses attention to boost the speed with which these models can be trained. The Transformer outperforms the Google Neur

In [14]:
# --------------------------------------------------------------
# 4. 최종 프롬프트(messages)를 사용해서 모델 출력 생성
# --------------------------------------------------------------
# pipe(messages):
#   - 지금까지 우리가 만든 messages (role + content 구조)를
#     Phi-3 모델에 넣어서 응답을 생성합니다.
#   - 내부적으로는:
#       1) tokenizer.apply_chat_template()로 ChatML 형식 프롬프트 생성
#       2) 토큰화 → 모델 추론 → 텍스트로 디코딩
#     과정을 한 번에 처리합니다.
outputs = pipe(messages)

# outputs는 리스트 형태로 결과를 반환합니다.
# outputs[0]         → 첫 번째 생성 결과(딕셔너리)
# outputs[0]["generated_text"] → 실제 생성된 요약 텍스트 부분만 꺼내기
print(outputs[0]["generated_text"])

 - The Transformer model, introduced in the paper "Attention is All You Need," utilizes attention mechanisms to enhance the training speed of deep learning models, particularly in neural machine translation.
- The model consists of an encoding component and a decoding component, each with multiple identical sub-layers. The encoding component includes a self-attention layer and a feed-forward neural network, while the decoding component includes a self-attention layer and a feed-forward neural network, with an additional attention layer between them.
- The model's architecture allows for parallelization, making it efficient for training on large datasets.
- The Transformer model is recommended by Google Cloud for use with their Cloud TPU offering, and TensorFlow and PyTorch implementations are available.

The Transformer model, introduced in the paper "Attention is All You Need," revolutionized the field of deep learning by utilizing attention mechanisms to enhance the training speed of

# 한국어 요약 버전 프롬프트

In [16]:
text = """이전 글에서는 현대 딥러닝 모델에서 보편적으로 사용되는 어텐션(Attention) 기법에 대해 살펴보았습니다. 어텐션은 신경망 기반 기계번역 모델의 성능을 높이는 데 크게 기여한 개념입니다. 이번 글에서는 어텐션을 활용해 학습 속도를 대폭 끌어올린 모델인 Transformer를 다룹니다. Transformer는 특정 작업에서 Google의 Neural Machine Translation 모델보다 뛰어난 성능을 보입니다. 그러나 Transformer의 가장 큰 장점은 모델이 병렬화(Parallelization)에 매우 잘 맞는 구조라는 점입니다. 실제로 Google Cloud TPU에서도 Transformer를 기본 참조 모델로 사용할 것을 권장하고 있습니다. 이제 이 모델이 어떻게 동작하는지 하나씩 분해해 살펴보겠습니다.

Transformer는 논문 Attention is All You Need에서 처음 제안되었습니다. TensorFlow 버전은 Tensor2Tensor 패키지로 제공되며, Harvard NLP group은 해당 논문에 대한 PyTorch 구현 가이드도 제공합니다. 여기에서는 내용을 지나치게 복잡하지 않게, 핵심 개념을 하나씩 단계적으로 설명하는 방식으로 접근해 보겠습니다.
전체적인 구조
기계번역 모델을 기준으로 Transformer를 하나의 블랙박스로 보면, 한 언어의 문장을 입력으로 받아 다른 언어의 문장을 출력합니다.
이 블랙박스를 열어보면 내부는 인코더(Encoder) 스택과 디코더(Decoder) 스택, 그리고 이 둘 사이를 연결하는 여러 경로로 구성됩니다.
인코더는 여러 개의 동일한 구조의 블록을 위로 쌓아 올린 형태(논문에서는 6개 사용, 개수는 조정 가능)
디코더도 동일한 수의 블록을 스택 형태로 구성
각 인코더는 서로 동일한 구조를 갖지만 가중치를 공유하지 않습니다. 인코더는 두 개의 서브 레이어로 구성됩니다.
Self-Attention Layer
입력 문장의 각 단어가 문장 내 다른 단어를 참고하며 자기 자신을 인코딩하는 레이어입니다. 이후 자세히 설명합니다.
Feed-Forward Neural Network
Self-Attention의 출력이 각 위치별로 동일한 신경망을 통과합니다.
디코더에도 두 레이어가 있지만, 그 중간에 **인코더의 출력을 참고하는 어텐션 레이어(Encoder–Decoder Attention)**가 추가로 들어 있습니다. 이는 기존 seq2seq 모델에서의 attention과 유사하게, 디코더가 입력 문장의 관련 부분에 집중할 수 있도록 돕습니다.
벡터 흐름 살펴보기
註: 아래는 학습된 모델에서 입력을 출력으로 변환하는 과정입니다.
NLP 작업에서는 먼저 입력되는 각 단어를 **워드 임베딩(embedding)**을 사용해 벡터로 변환합니다.
각 단어는 길이 512의 벡터로 임베딩됩니다.
이 임베딩 과정은 첫 번째(가장 아래) 인코더에서만 수행됩니다. 그 위의 인코더들은 모두 동일한 크기의 벡터 리스트를 입력으로 받는데,
첫 번째 인코더는 단어 임베딩을 받고
그 위 인코더들은 아래 인코더의 출력을 입력받습니다.
이 벡터 리스트의 길이는 최대 문장 길이에 따라 결정되는 하이퍼파라미터입니다.
임베딩된 단어들은 인코더의 두 레이어(Self-Attention → Feed-Forward)를 통과합니다.
Transformer의 핵심 특징 중 하나는,
각 위치의 단어가 자기 경로를 따라 독립적으로 처리된다는 점입니다.
Self-Attention에서는 단어 간의 상호참조가 일어나므로 경로가 서로 의존함
그러나 Feed-Forward 레이어에서는 위치별 계산이 완전히 독립적이어서 병렬 처리가 가능합니다 → 학습 속도 향상
앞서 설명했듯이 인코더는 벡터 리스트를 입력으로 받아,
Self-Attention Layer
Feed-Forward Network
을 통과시킨 뒤
그 결과를 위쪽 인코더로 넘깁니다.
이러한 구조가 반복되며 입력 문장의 의미가 점차 더 높은 수준의 표현으로 변환됩니다."""

In [18]:
persona = (
    "당신은 대형언어모델(LLM)과 딥러닝 논문을 명확하게 정리하는 전문가입니다. "
    "어려운 논문도 핵심만 뽑아 이해하기 쉬운 방식으로 설명하는 능력이 뛰어납니다.\n"
)

instruction = (
    "아래에 제공된 텍스트의 핵심 내용을 요약하고 한국어로 번역해 주세요.\n"
)

context = (
    "요약은 연구자가 빠르게 내용을 파악할 수 있도록, 가장 중요한 개념과 기여점을 중심으로 정리해야 합니다. "
    "불필요한 배경 설명은 최소화하고, 논문의 목적·방법·결과·의의를 우선적으로 다뤄 주세요.\n"
)

data_format = (
    "출력 형식은 다음과 같습니다:\n"
    "1) 핵심 내용을 불릿 포인트로 구조화하여 요약\n"
    "2) 이어서 주요 결과와 의미를 3~4문장으로 정리한 짧은 단락\n"
    "3) 한국어로 번역하여 출력\n"
)

audience = (
    "요약의 대상은 바쁜 연구자들입니다. "
    "최신 연구 경향을 빠르게 파악하는 데 도움이 되는 요약을 제공해야 합니다.\n"
)

tone = (
    "전문적이고 명확한 톤으로 작성해 주세요.\n"
)

data = f"요약할 텍스트:\n{text}"

query = persona + instruction + context + data_format + audience + tone + data

In [19]:
# --------------------------------------------------------------
# 3. 프롬프트를 Phi-3(ChatML) 형식으로 자동 변환
# --------------------------------------------------------------
# messages:
#   - Chat 모델이 요구하는 입력 형식: role + content 구조
#   - 여기서는 전체 query(프롬프트)를 user 역할로 전달합니다.
messages = [
    {"role": "user", "content": query}
]

# Chat 템플릿 적용:
#   - tokenizer.apply_chat_template():
#       messages 구조를 Phi-3 모델이 읽는 내부 포맷(ChatML)으로 자동 변환
#       (예: <|user|> ... <|assistant|> 형태)
#
#   - tokenize=False:
#       토큰 ID로 변환하지 않고, 단순 문자열로 확인만 하는 옵션
#       디버깅에 매우 유용 → 모델이 실제로 어떤 프롬프트를 입력받는지 눈으로 볼 수 있음
print(tokenizer.apply_chat_template(messages, tokenize=False))

<|user|>
당신은 대형언어모델(LLM)과 딥러닝 논문을 명확하게 정리하는 전문가입니다. 어려운 논문도 핵심만 뽑아 이해하기 쉬운 방식으로 설명하는 능력이 뛰어납니다.
아래에 제공된 텍스트의 핵심 내용을 요약하고 한국어로 번역해 주세요.
요약은 연구자가 빠르게 내용을 파악할 수 있도록, 가장 중요한 개념과 기여점을 중심으로 정리해야 합니다. 불필요한 배경 설명은 최소화하고, 논문의 목적·방법·결과·의의를 우선적으로 다뤄 주세요.
출력 형식은 다음과 같습니다:
1) 핵심 내용을 불릿 포인트로 구조화하여 요약
2) 이어서 주요 결과와 의미를 3~4문장으로 정리한 짧은 단락
3) 한국어로 번역하여 출력
요약의 대상은 바쁜 연구자들입니다. 최신 연구 경향을 빠르게 파악하는 데 도움이 되는 요약을 제공해야 합니다.
전문적이고 명확한 톤으로 작성해 주세요.
요약할 텍스트:
이전 글에서는 현대 딥러닝 모델에서 보편적으로 사용되는 어텐션(Attention) 기법에 대해 살펴보았습니다. 어텐션은 신경망 기반 기계번역 모델의 성능을 높이는 데 크게 기여한 개념입니다. 이번 글에서는 어텐션을 활용해 학습 속도를 대폭 끌어올린 모델인 Transformer를 다룹니다. Transformer는 특정 작업에서 Google의 Neural Machine Translation 모델보다 뛰어난 성능을 보입니다. 그러나 Transformer의 가장 큰 장점은 모델이 병렬화(Parallelization)에 매우 잘 맞는 구조라는 점입니다. 실제로 Google Cloud TPU에서도 Transformer를 기본 참조 모델로 사용할 것을 권장하고 있습니다. 이제 이 모델이 어떻게 동작하는지 하나씩 분해해 살펴보겠습니다.

Transformer는 논문 Attention is All You Need에서 처음 제안되었습니다. TensorFlow 버전은 Tensor2Tensor 패키지로 제공되며, Harvard NLP group은 해당 논문에 대한 PyTorch 구현 가이드도 제공합니다. 여기에서는 내용

In [20]:
# --------------------------------------------------------------
# 4. 최종 프롬프트(messages)를 사용해서 모델 출력 생성
# --------------------------------------------------------------
# pipe(messages):
#   - 지금까지 우리가 만든 messages (role + content 구조)를
#     Phi-3 모델에 넣어서 응답을 생성합니다.
#   - 내부적으로는:
#       1) tokenizer.apply_chat_template()로 ChatML 형식 프롬프트 생성
#       2) 토큰화 → 모델 추론 → 텍스트로 디코딩
#     과정을 한 번에 처리합니다.
outputs = pipe(messages)

# outputs는 리스트 형태로 결과를 반환합니다.
# outputs[0]         → 첫 번째 생성 결과(딕셔너리)
# outputs[0]["generated_text"] → 실제 생성된 요약 텍스트 부분만 꺼내기
print(outputs[0]["generated_text"])

 1. Transformer는 논문 "Attention is All You Need"에서 제안되었습니다.
2. TensorFlow 버전에서는 Tensor2Tensor 패키지로 제공되며, 논문에 대한 PyTorch 구현도 제공합니다.
3. 따라서 본 블랙박스는 인코더(Encoder)와 디코더(Decoder) 구성 요소로 이루어져 있습니다.
4. 인코더는 6개의 동일한 구조의 블록을 위로 쌓습니다.
5. 디코더도 6개의 동일한 구조의 블록을 스택 형태로 구성합니다.
6. 인코더는 서로 동일한 구조를 갖지만 가중치를 공유하지 않습니다.
7. 인코더는 두 개의 서브 레이어로 구성됩니다.
8. Self-Attention Layer는 입력 문장의 단어들이 서로 참조하며 자기 자신을 인코딩합니다.
9. Feed-Forward Neural Network은 Self-Attention의 출력을 각 위치별로 동일한 신경망을 통과합니다.
10. 디코더에서도 인�


## One-shot Prompting
- 문맥 내 학습: 예시 제공

In [21]:
# --------------------------------------------------------------
# 가상의 단어가 들어간 문장을 만들어내는 one-shot 프롬프트 예제
# --------------------------------------------------------------
# one_shot_prompt:
#   - role=user / role=assistant 구조를 번갈아 넣어서
#     모델에 “이런 형식으로 답해야 한다”는 예시(example)를 제공하는 방식입니다.
#
#   - 첫 번째 사용자 메시지:
#       새로운 단어(Gigamuru)가 무엇인지 설명하고
#       "이 단어가 들어간 문장의 예시를 알려 달라"고 요청합니다.
#
#   - 두 번째 assistant 메시지:
#       모델이 참고할 수 있도록 'Gigamuru'를 사용한 문장 예시를 제공합니다.
#
#   - 세 번째 사용자 메시지:
#       또 다른 가상의 단어(screeg)에 대해
#       "이 단어가 들어간 문장 예시를 만들어 달라"고 요청합니다.
#
# 이렇게 예제를 한 번 보여준 뒤(=one-shot),
# 모델이 같은 패턴을 따라 새로운 답을 생성하도록 유도합니다.
one_shot_prompt = [
    {
        "role": "user",
        "content": (
            "A 'Gigamuru' is a type of Japanese musical instrument. "
            "An example of a sentence that uses the word Gigamuru is:"
        )
    },
    {
        "role": "assistant",
        "content": (
            "I have a Gigamuru that my uncle gave me as a gift. "
            "I love to play it at home."
        )
    },
    {
        "role": "user",
        "content": (
            "To 'screeg' something is to swing a sword at it. "
            "An example of a sentence that uses the word screeg is:"
        )
    }
]

# --------------------------------------------------------------
# apply_chat_template():
#   - 위의 role/user·assistant 구조 메시지를
#     모델이 실제로 이해하는 ChatML 형태의 프롬프트로 자동 변환해 줍니다.
#   - tokenize=False → 텍스트로만 확인 (토큰 ID 변환 X)
# --------------------------------------------------------------
print(tokenizer.apply_chat_template(one_shot_prompt, tokenize=False))

<|user|>
A 'Gigamuru' is a type of Japanese musical instrument. An example of a sentence that uses the word Gigamuru is:<|end|>
<|assistant|>
I have a Gigamuru that my uncle gave me as a gift. I love to play it at home.<|end|>
<|user|>
To 'screeg' something is to swing a sword at it. An example of a sentence that uses the word screeg is:<|end|>
<|endoftext|>


In [22]:
# --------------------------------------------------------------
# one_shot_prompt를 모델에 넣어서 실제 출력을 생성하는 단계
# --------------------------------------------------------------
# pipe(one_shot_prompt):
#   - 지금까지 만든 one-shot 예시(가상의 단어 예문)를 기반으로
#     모델이 이어서 문장을 생성하도록 요청합니다.
#
#   - 첫 번째 가상의 단어(Gigamuru)는 예시를 주었고,
#     두 번째 가상의 단어(screeg)에 대해
#     모델이 같은 방식으로 문장을 만들어서 답하게 됩니다.
#
# outputs:
#   - 모델이 생성한 결과를 리스트 형태로 반환합니다.
#   - outputs[0]["generated_text"] → 생성된 문장만 뽑아 출력.
outputs = pipe(one_shot_prompt)

# 모델이 생성한 문장을 화면에 출력
print(outputs[0]["generated_text"])

 During the medieval reenactment, the knight skillfully screeged the wooden target, impressing the onlookers with his prowess.


### One-shot Prompting 한국어 예제

In [23]:
# --------------------------------------------------------------
# 가상의 한국어 단어가 들어간 문장을 만들어내는 one-shot 프롬프트 예제
# --------------------------------------------------------------
# one_shot_prompt:
#   - role=user / role=assistant 메시지를 번갈아 넣어
#     모델이 “이런 형식으로 대답해야 한다”는 예시 패턴을 학습하도록 돕는 방식입니다.
#
#   - 첫 번째 사용자 메시지:
#       새로운 가상의 단어(누리솜)의 의미를 설명하고
#       “이 단어가 들어간 문장 예시를 알려 달라”고 요청합니다.
#
#   - 두 번째 assistant 메시지:
#       모델이 참고할 수 있도록 '누리솜'을 포함한 예시 문장을 제공합니다.
#
#   - 세 번째 사용자 메시지:
#       또 다른 가상의 동사(다올하다)의 뜻을 설명하고
#       “이 동사가 들어간 문장을 만들어 달라”고 요청합니다.
#
# 이렇게 한 번의 예시(one-shot)를 보여준 뒤,
# 모델이 같은 패턴을 따라 새로운 답을 생성하도록 유도합니다.
# --------------------------------------------------------------

one_shot_prompt = [
    {
        "role": "user",
        "content": (
            "'누리솜'은 따뜻하고 포근한 느낌을 주는 상상의 천을 뜻하는 단어입니다. "
            "이 단어가 들어간 예시 문장은 다음과 같습니다:"
        )
    },
    {
        "role": "assistant",
        "content": (
            "어머니가 만들어 주신 누리솜 담요는 추운 밤마다 큰 위로가 됩니다."
        )
    },
    {
        "role": "user",
        "content": (
            "'다올하다'는 마음을 다해 정성껏 준비한다는 뜻을 가진 가상의 동사입니다. "
            "이 단어가 들어간 문장의 예시는:"
        )
    }
]

# --------------------------------------------------------------
# apply_chat_template():
#   - 위의 role/user·assistant 구조 메시지를
#     모델이 실제로 이해하는 ChatML 형태 프롬프트로 자동 변환합니다.
#   - tokenize=False → 텍스트로만 확인 (토큰 ID 변환 없음)
# --------------------------------------------------------------

print(tokenizer.apply_chat_template(one_shot_prompt, tokenize=False))

<|user|>
'누리솜'은 따뜻하고 포근한 느낌을 주는 상상의 천을 뜻하는 단어입니다. 이 단어가 들어간 예시 문장은 다음과 같습니다:<|end|>
<|assistant|>
어머니가 만들어 주신 누리솜 담요는 추운 밤마다 큰 위로가 됩니다.<|end|>
<|user|>
'다올하다'는 마음을 다해 정성껏 준비한다는 뜻을 가진 가상의 동사입니다. 이 단어가 들어간 문장의 예시는:<|end|>
<|endoftext|>


In [24]:
# --------------------------------------------------------------
# one_shot_prompt를 모델에 넣어서 실제 출력을 생성하는 단계
# --------------------------------------------------------------
# pipe(one_shot_prompt):
#   - 지금까지 만든 one-shot 예시(가상의 단어 예문)를 기반으로
#     모델이 이어서 문장을 생성하도록 요청합니다.
#
#   - 첫 번째 가상의 단어(Gigamuru)는 예시를 주었고,
#     두 번째 가상의 단어(screeg)에 대해
#     모델이 같은 방식으로 문장을 만들어서 답하게 됩니다.
#
# outputs:
#   - 모델이 생성한 결과를 리스트 형태로 반환합니다.
#   - outputs[0]["generated_text"] → 생성된 문장만 뽑아 출력.
outputs = pipe(one_shot_prompt)

# 모델이 생성한 문장을 화면에 출력
print(outputs[0]["generated_text"])

 그는 다올하다 예산을 준비했습니다. 그러나 그는 예산이 없었으므로 다올하다를 못했습니다. 그러나 다올하다를 못한 예산을 복구하려면 새로운 예산을 찾아야 합니다. 새로운 예산을 찾았으며, 그는 다올하다 예산을 준비했습니다. 이제 다올하다를 못 했습니다. 다올하다를 못 했으므로 다올하다를 다시 시작하기 위해 새로운 계획을 세워야 합니다. 그러나 다올하다를 다시 시작하기 전에 새로운 계획을 세워야 합니다. 다올하다를 다시 시작하기 전에 새로운 계획을 세워야 합니다. 그러나 다올하다를 다시 시작하기 전에 새로운 계획을 세워야 합니다. 그러나 다올하다를 다시 시작하기 전에 새로운 ����


## Prompt Chaining(프롬프트 체인): 문제 쪼개기


### ✔︎ 코드의 전체 흐름 요약

1. 제품 이름 + 슬로건 생성

2. 그 출력값을 product_description 변수에 저장

3. product_description을 “홍보 문구 생성 프롬프트”에 삽입

4. pipe()로 실행해 짧은 sales pitch 생성

이렇게 하면 자연스럽게:

  - 제품 이름

  - 슬로건

  - 짧은 광고 문구

가 맞물려 이어지는 형태의 콘텐츠를 만들 수 있습니다.

In [25]:
# --------------------------------------------------------------
# 제품 이름과 슬로건 생성하기
# --------------------------------------------------------------
# product_prompt:
#   - 사용자 메시지(role=user)를 하나 넣어,
#     “LLM을 활용한 챗봇의 이름과 슬로건을 만들어 달라”고 요구합니다.
#
#   - 파이프라인(pipe)이 이 요청을 받아 창의적인 제품 이름과 슬로건을 생성하게 됩니다.
product_prompt = [
    {"role": "user", "content": "Create a name and slogan for a chatbot that leverages LLMs."}
]

# --------------------------------------------------------------
# 모델에 프롬프트 전달하여 출력 생성
# --------------------------------------------------------------
# pipe(product_prompt):
#   - 메시지 형태의 입력을 받아 ChatML 템플릿 → 토큰화 → 추론 → 디코딩 과정을 수행
#   - 결과를 리스트 형태로 반환합니다.
outputs = pipe(product_prompt)

# --------------------------------------------------------------
# 생성된 텍스트만 추출
# --------------------------------------------------------------
product_description = outputs[0]["generated_text"]

# 생성 결과 출력
print(product_description)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


 Name: ChatSage
Slogan: "Unleashing the power of AI to enhance your conversations."


In [26]:
# --------------------------------------------------------------
# 제품 이름 + 슬로건 → 아주 짧은 홍보 문구 생성하기
# --------------------------------------------------------------

# sales_prompt:
#   - 앞 단계에서 만든 product_description(제품 이름 + 슬로건)을
#     한 문장으로 요약한 “sales pitch” 생성 요청에 넣습니다.
#
#   - f-string을 사용하여 product_description을 문자열 안에 삽입합니다.
sales_prompt = [
    {
        "role": "user",
        "content": (
            f"Generate a very short sales pitch for the following product: "
            f"'{product_description}'"
        )
    }
]

# --------------------------------------------------------------
# 모델 실행: 짧은 홍보 문구 생성
# --------------------------------------------------------------
outputs = pipe(sales_prompt)

# --------------------------------------------------------------
# 생성된 홍보 문구만 추출
# --------------------------------------------------------------
sales_pitch = outputs[0]["generated_text"]

# 출력 확인
print(sales_pitch)

 Introducing ChatSage, the revolutionary AI-powered tool designed to elevate your conversations to new heights. With our cutting-edge technology, we unleash the power of AI to enhance your interactions, making every conversation more engaging, insightful, and meaningful. Experience the future of communication with ChatSage today!


프롬프트 체이닝 한국어 예제

In [27]:
# --------------------------------------------------------------
# 제품 이름과 슬로건 생성하기 (한국어 버전)
# --------------------------------------------------------------
# product_prompt:
#   - 사용자 메시지(role=user)를 하나 넣어
#     “LLM 기반 챗봇의 이름과 슬로건을 만들어 달라”고 요청합니다.
#
#   - 파이프라인(pipe)은 이 요청을 받아
#     창의적인 한국어 제품명과 슬로건을 생성하게 됩니다.
product_prompt = [
    {"role": "user", "content": "LLM을 활용한 한국어 챗봇의 제품 이름과 슬로건을 만들어 주세요."}
]

# --------------------------------------------------------------
# 모델에 프롬프트 전달하여 출력 생성
# --------------------------------------------------------------
# pipe(product_prompt):
#   - 메시지 형태의 입력을 ChatML 템플릿으로 변환한 뒤
#     토큰화 → 추론 → 디코딩 과정을 수행하여
#     생성된 텍스트를 리스트 형태로 반환합니다.
outputs = pipe(product_prompt)

# --------------------------------------------------------------
# 생성된 텍스트만 추출
# --------------------------------------------------------------
product_description = outputs[0]["generated_text"]

# 생성 결과 출력
print(product_description)

 제품 이름: 한글챗봇 파이썬

슬로건:

안녕하세요, 저는 한글챗봇 파이썬입니다. 저는 한국어 챗봇 기능을 제공하고 있습니다. 챗봇 기능은 사용자의 질문을 받아 정확한 답변을 제공하는 기능입니다. 저는 토픽 모델링, 자연어 처리, 그리고 챗봇 기능을 활용하여 최신 기술을 사용하고 있습니다. 저는 특정 챗봇 기능을 추가하거나 변경하려면 저에게 입력을 주시면 됩니다. 저는 항상 항상 최신 기술을 사용하여 챗봇 기능을 개선하고 있습니다. 저는 챗봇 기능을 사용하여 사용자의 질문을 받아 정확한 답변을 제공하는 것을 목표로 하고 있습니다. 저는 챗봇 기능을 사용하여 사용자의 �


In [28]:
# --------------------------------------------------------------
# 제품 이름 + 슬로건 → 아주 짧은 홍보 문구 생성하기 (한국어 버전)
# --------------------------------------------------------------

# sales_prompt:
#   - 앞 단계에서 생성된 product_description(제품 이름 + 슬로건)을
#     한 문장으로 요약한 “짧은 홍보 문구”를 만들어 달라고 요청합니다.
#
#   - f-string을 사용하여 product_description을 메시지 안에 삽입합니다.
sales_prompt = [
    {
        "role": "user",
        "content": (
            f"다음 제품 소개 내용을 기반으로 아주 짧은 한국어 홍보 문구를 만들어 주세요: "
            f"'{product_description}'"
        )
    }
]

# --------------------------------------------------------------
# 모델 실행: 짧은 홍보 문구 생성
# --------------------------------------------------------------
outputs = pipe(sales_prompt)

# --------------------------------------------------------------
# 생성된 홍보 문구만 추출
# --------------------------------------------------------------
sales_pitch = outputs[0]["generated_text"]

# 출력 확인
print(sales_pitch)

 안녕하세요! 한글챗봇 파이썬입니다. 저는 한국어 챗봇 기능을 제공합니다. 챗봇 기능은 사용자의 질문을 받아 정확한 답변을 제공합니다. 저는 토픽 모델링, 자연어 처리, 그리고 챗봇 기능을 활용하여 최신 기술을 사용합니다. 저는 특정 챗봇 기능을 추가하거나 변경하려면 저에게 입력을 주시면 됩니다. 저는 항상 최신 기술을 사용하여 챗봇 기능을 개선하고 있습니다. 저는 챗봇 기능을 사용하여 사용자의 질문을 받아 정확한 답변을 제공하는 것을 목표로 하고 있습니다. 저는 챗봇 기능을 사용하여 사용자의 질문을 받아 정확한 답변을 제공합니다.


프롬프트 체이닝 중국어 예제

In [29]:
# --------------------------------------------------------------
# 生成产品名称和口号（中文版本）
# --------------------------------------------------------------
# product_prompt:
#   - 只加入一条用户消息(role=user)，
#     要求模型“为一个使用 LLM 的聊天机器人创建名称和口号”。
#
#   - 管道(pipe)会根据该请求生成富有创意的产品名称和口号。
product_prompt = [
    {"role": "user", "content": "请为一个基于大型语言模型的聊天机器人创建一个名称和口号。"}
]

# --------------------------------------------------------------
# 将提示词发送给模型并生成输出
# --------------------------------------------------------------
# pipe(product_prompt):
#   - 接受消息格式输入，自动完成 ChatML 模板转换 → 分词 → 推理 → 解码
#   - 并以列表形式返回结果。
outputs = pipe(product_prompt)

# --------------------------------------------------------------
# 只提取生成的文本内容
# --------------------------------------------------------------
product_description = outputs[0]["generated_text"]

# 输出结果
print(product_description)

 名称：智慧聊天仙

口号：“您的智慧，我的般”


这个名称和口号强调了聊天机器人的智能特性，同时借助于仙界的神秘感，吸引用户的好奇心和兴趣。


In [30]:
# --------------------------------------------------------------
# 产品名称 + 口号 → 生成一句超短宣传语（中文版本）
# --------------------------------------------------------------

# sales_prompt:
#   - 将上一阶段生成的 product_description（产品名称 + 口号）
#     放入请求中，要求模型基于其内容生成一句超短宣传语。
#
#   - 使用 f-string 将 product_description 插入字符串中。
sales_prompt = [
    {
        "role": "user",
        "content": (
            f"请根据以下产品介绍，生成一句非常简短的中文宣传语："
            f"‘{product_description}’"
        )
    }
]

# --------------------------------------------------------------
# 执行模型：生成超短宣传语
# --------------------------------------------------------------
outputs = pipe(sales_prompt)

# --------------------------------------------------------------
# 只提取生成的宣传语内容
# --------------------------------------------------------------
sales_pitch = outputs[0]["generated_text"]

# 输出结果
print(sales_pitch)

 智慧聊天仙，“您的智慧，我的般”，让您与仙界交流智慧。


# 생성 모델을 사용한 추론


### CoT: 응답하기 전에 생각하기


### ✔︎ 이 프롬프트의 핵심 아이디어

모델에게 직접 “추론하지 말고 답만 말해”라고 지시하지 않아도,

예시(answer-only format)를 한 번 보여주면 모델이 그 패턴을 따라간다.

즉,
one-shot imitation learning을 이용한 “답만 출력하기” 방식입니다.

In [31]:
# --------------------------------------------------------------
# 추론 과정 없이 '정답만' 내도록 유도하는 one-shot 프롬프트 예제
# --------------------------------------------------------------
# standard_prompt:
#   - 첫 번째 user 메시지: 간단한 산수 문제 제시
#   - 첫 번째 assistant 메시지: 정답만 제시 ("11")
#
#   → 이 예시가 모델에게 “이런 형식으로 답하라”고 가르치는 역할(one-shot 학습)
#
#   - 두 번째 user 메시지에는 새로운 문제를 주고,
#     모델이 같은 방식(=추론 없이 숫자만 답하기)으로 응답하도록 유도합니다.
standard_prompt = [
    {
        "role": "user",
        "content": (
            "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. "
            "Each can has 3 tennis balls. How many tennis balls does he have now?"
        )
    },
    {
        "role": "assistant",
        "content": "11"   # 예시 정답 → 모델에게 패턴을 가르침
    },
    {
        "role": "user",
        "content": (
            "The cafeteria had 25 apples. If they used 20 to make lunch "
            "and bought 6 more, how many apples do they have?"
        )
    }
]

# --------------------------------------------------------------
# 모델이 one-shot 예시 패턴을 그대로 따라 숫자만 출력하도록 생성
# --------------------------------------------------------------
outputs = pipe(standard_prompt)

# 생성된 정답(숫자)
print(outputs[0]["generated_text"])

 The cafeteria started with 25 apples. They used 20 apples to make lunch, so they had 25 - 20 = 5 apples left. After buying 6 more apples, they now have 5 + 6 = 11 apples.


In [35]:
# --------------------------------------------------------------
# 추론 과정 없이 '정답만' 출력하도록 유도하는 one-shot 프롬프트 (한국어 버전)
# --------------------------------------------------------------
# standard_prompt:
#   - 첫 번째 user 메시지: 간단한 산수 문제 제시
#   - 첫 번째 assistant 메시지: 정답만 제시 ("11")
#
#   → 이 예시가 모델에게 “이런 형식으로 답하라”(=추론 생략, 숫자만 출력)를 가르치는 역할
#
#   - 두 번째 user 메시지: 새로운 문제 제시
#     모델은 첫 예시의 패턴을 따라, 계산 과정 없이 숫자만 답하도록 유도됨.
# --------------------------------------------------------------

standard_prompt = [
    {
        "role": "user",
        "content": (
            "민수에게 테니스공이 5개 있습니다. 그는 테니스공이 3개씩 들어 있는 "
            "캔 2개를 더 샀습니다. 지금 민수는 테니스공을 모두 몇 개 가지고 있을까요?"
        )
    },
    {
        "role": "assistant",
        "content": "11"   # 예시 정답 → 모델에게 답변 패턴을 학습시킴
    },
    {
        "role": "user",
        "content": (
            "급식실에 사과가 25개 있었습니다. 점심 준비에 20개를 사용하고 "
            "다시 6개를 샀다면, 지금 사과는 몇 개 남았을까요?"
        )
    }
]

# --------------------------------------------------------------
# 모델이 one-shot 예시 패턴을 따라 숫자만 출력하도록 생성
# --------------------------------------------------------------
outputs = pipe(standard_prompt)

# 생성된 정답(숫자)
print(outputs[0]["generated_text"])

 9


### ✔︎ CoT(Chain-of-Thought)가 무엇인가?

- 모델이 추론 과정을 단계별로 말하며 문제를 해결하는 방식

- 사람의 생각 흐름처럼 “근거 → 계산 → 결론” 순으로 설명

- 단순 정답만 주는 방식보다 더 정확하고 안정적

In [32]:
# --------------------------------------------------------------
# Chain-of-Thought(CoT) 방식으로 답하도록 유도하는 one-shot 프롬프트
# --------------------------------------------------------------
# cot_prompt:
#   - 첫 번째 user 메시지: 산수 문제 제시
#   - 첫 번째 assistant 메시지:
#         정답만 말하는 것이 아니라
#         "단계별 추론(=CoT)"을 보여주는 예시를 포함합니다.
#
#     예시:
#       Roger started with 5 balls.
#       2 cans × 3 balls = 6 balls.
#       5 + 6 = 11.
#       So the answer is 11.
#
#   - 두 번째 user 메시지:
#       새로운 문제를 제시하면
#       모델은 위 CoT 패턴을 따라서
#       단계별 추론을 먼저 하고 → 그다음 정답을 말하게 됩니다.
cot_prompt = [
    {
        "role": "user",
        "content": (
            "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. "
            "Each can has 3 tennis balls. How many tennis balls does he have now?"
        )
    },
    {
        "role": "assistant",
        "content": (
            "Roger started with 5 balls. 2 cans of 3 tennis balls each is 6 tennis balls. "
            "5 + 6 = 11. The answer is 11."
        )
    },
    {
        "role": "user",
        "content": (
            "The cafeteria had 23 apples. If they used 20 to make lunch "
            "and bought 6 more, how many apples do they have?"
        )
    }
]

# --------------------------------------------------------------
# 모델이 CoT 예시 패턴을 따라 단계별 추론을 생성
# --------------------------------------------------------------
outputs = pipe(cot_prompt)

# 생성된 단계별 추론 + 정답 출력
print(outputs[0]["generated_text"])

 The cafeteria started with 23 apples. They used 20 apples for lunch, so they had 23 - 20 = 3 apples left. After buying 6 more apples, they now have 3 + 6 = 9 apples. The answer is 9.


In [39]:
# --------------------------------------------------------------
# Chain-of-Thought(CoT) 방식으로 답하도록 유도하는 one-shot 프롬프트 (한국어 버전)
# --------------------------------------------------------------
# cot_prompt:
#   - 첫 번째 user 메시지: 산수 문제 제시
#   - 첫 번째 assistant 메시지:
#       정답만 말하는 것이 아니라,
#       단계별 추론(CoT)을 포함한 예시를 제공합니다.
#
#     예시 CoT:
#       민수는 처음에 테니스공이 5개 있었다.
#       테니스공이 3개씩 들어 있는 캔 2개를 사면 6개를 추가로 얻게 된다.
#       5 + 6 = 11.
#       따라서 정답은 11이다.
#
#   - 두 번째 user 메시지:
#       새로운 문제를 제시하면
#       모델은 위 CoT 패턴을 따라
#       단계별 추론 → 최종 답을 생성합니다.
# --------------------------------------------------------------

cot_prompt = [
    {
        "role": "user",
        "content": (
            "민수에게 테니스공이 5개 있습니다. 그는 테니스공이 3개씩 들어 있는 "
            "캔 2개를 더 샀습니다. 민수가 지금 가진 테니스공은 모두 몇 개인가요?"
        )
    },
    {
        "role": "assistant",
        "content": (
            "민수는 처음에 테니스공이 5개 있었다. "
            "테니스공이 3개씩 들어 있는 캔을 2개 사면 6개를 더 얻게 된다. "
            "5 + 6 = 11. 따라서 정답은 11이다."
        )
    },
    {
        "role": "user",
        "content": (
            "급식실에 사과가 23개 있었습니다. 점심을 만드는 데 20개를 사용했고, "
            "6개를 새로 샀다면, 지금 사과는 몇 개인가요?"
        )
    }
]

# --------------------------------------------------------------
# 모델이 CoT 예시 패턴을 따라 단계별 추론을 생성
# --------------------------------------------------------------
outputs = pipe(cot_prompt)

# 생성된 단계별 추론 + 정답 출력
print(outputs[0]["generated_text"])

 급식실에 초기에 23개의 사과가 있었습니다. 점심을 만들면 20개가 사용되었으므로, 남은 사과는 23 - 20 = 3개가 남았다. 그런 다음 6개를 새로 샀다면, 남은 사과는 3 + 6 = 9개가 되었다. 따라서 지금 사과는 9개 입니다.


## 제로-샷 CoT

### ✔︎ Zero-Shot CoT란 무엇인가?

- 예시(CoT demonstration)를 제공하지 않고

- 단순히 “Let's think step-by-step.” 같은 표현을 넣는 것만으로

- 모델이 스스로 단계별 추론을 시작함

- 강력한 CoT 유도 프롬프트 기법 중 하나

| 특징          | 설명                                                  |
| ----------- | --------------------------------------------------- |
| 예시 제공 필요 없음 | one-shot / few-shot 없이도 CoT 수행                      |
| 유도 문구로 충분   | “Let's think step by step”, “Let's reason it out” 등 |
| 추론 품질 향상    | 정답률이 즉시 올라가는 경우가 많음                                 |



In [40]:
# --------------------------------------------------------------
# Zero-Shot CoT (단 한 줄 힌트로 단계별 추론 유도)
# --------------------------------------------------------------
# zeroshot_cot_prompt:
#   - 예시(example)를 주지 않고,
#     "Let's think step-by-step." 라는 문장만 추가해서
#     모델에게 스스로 Chain-of-Thought 추론을 하도록 유도하는 방식입니다.
#
#   - 즉, one-shot 예시 없이도
#     “단계별로 생각해 보자”라는 지시만으로 CoT 생성이 가능해지는 기법.
zeroshot_cot_prompt = [
    {
        "role": "user",
        "content": (
            "The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, "
            "how many apples do they have? Let's think step-by-step."
        )
    }
]

# --------------------------------------------------------------
# 모델이 제로샷 CoT 방식으로 단계별 추론을 생성
# --------------------------------------------------------------
outputs = pipe(zeroshot_cot_prompt)

# 생성된 단계별 추론 결과 출력
print(outputs[0]["generated_text"])

 Step 1: Start with the initial number of apples in the cafeteria, which is 23.

Step 2: Subtract the number of apples used to make lunch, which is 20.
23 - 20 = 3 apples remaining.

Step 3: Add the number of apples bought, which is 6.
3 + 6 = 9 apples.

So, the cafeteria now has 9 apples.


In [41]:
# --------------------------------------------------------------
# Zero-Shot CoT (단 한 줄 힌트로 단계별 추론 유도)
# --------------------------------------------------------------
# zeroshot_cot_prompt:
#   - 예시(example)를 제공하지 않고,
#     "단계별로 생각해 보자."라는 문장을 추가하여
#     모델이 스스로 Chain-of-Thought 방식으로 추론하도록 유도하는 기법입니다.
#
#   - 즉, one-shot 예시 없이도
#     “단계별로 생각해 보자”라는 문장만으로 CoT 추론을 생성하도록 만드는 방식입니다.
zeroshot_cot_prompt = [
    {
        "role": "user",
        "content": (
            "급식실에 사과가 23개 있었습니다. 점심을 준비하는 데 20개를 사용하고, "
            "다시 6개를 구매했다면 사과는 몇 개가 남았을까요? 단계별로 생각해 보자."
        )
    }
]

# --------------------------------------------------------------
# 모델이 Zero-Shot CoT 방식으로 단계별 추론을 생성
# --------------------------------------------------------------
outputs = pipe(zeroshot_cot_prompt)

# 생성된 단계별 추론 출력
print(outputs[0]['generated_text'])

 초기 사과 수: 23개

점심 사용: -20개

다시 구매: +6개


계산:

23개 - 20개 = 3개 (점심 사용 후)

3개 + 6개 = 9개 (다시 구매 후)


따라서, 급식실에 남은 사과는 9개입니다.


## ToT: 중간 단계 탐색

### ✔︎ Zero-Shot ToT vs Zero-Shot CoT 차이

| 기법            | 설명                             | 특징                         |
| ------------- | ------------------------------ | -------------------------- |
| Zero-Shot CoT | “한 사람이 단계별로 생각하는 방식”           | `Let's think step-by-step` |
| Zero-Shot ToT | “여러 전문가가 동시에 아이디어를 내고 비교하는 방식” | 더 다양한 추론 경로·창의성·정확도 증가     |



In [42]:
# --------------------------------------------------------------
# Zero-Shot ToT (Tree-of-Thought) 프롬프트
# --------------------------------------------------------------
# ToT(Tree of Thought)란?
#   - 한 명이 순차적으로 생각하는 CoT와 다르게
#     "여러 명의 전문가가 동시에 단계별로 생각을 제시"하고
#     그중 가장 합리적인 생각으로 수렴하도록 유도하는 기법입니다.
#
#   - 논리적 문제 해결, 창의적 문제 해결 등에 효과적입니다.
#
# 아래 프롬프트는:
#   - 3명의 전문가가 있다고 가정
#   - 각 전문가는 1단계씩 생각을 공유
#   - 스스로 오류를 발견하면 토론에서 빠져나감
#   - 최종적으로 그룹이 토론 결과를 모아 결론을 도출
#
# 즉, 모델이 "여러 개의 생각 경로(Thoughts)를 생성 → 비교 → 수렴"하는
# Tree-of-Thought 구조를 스스로 시도하도록 유도하는 제로샷 프롬프트입니다.
zeroshot_tot_prompt = [
    {
        "role": "user",
        "content": (
            "Imagine three different experts are answering this question. "
            "All experts will write down 1 step of their thinking, then share it with the group. "
            "Then all experts will go on to the next step, etc. "
            "If any expert realises they're wrong at any point then they leave. "
            "The question is 'The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, "
            "how many apples do they have?' "
            "Make sure to discuss the results."
        )
    }
]

In [43]:
# --------------------------------------------------------------
# 5. Zero-Shot ToT 프롬프트 실행하여 결과 생성
# --------------------------------------------------------------
# pipe(zeroshot_tot_prompt):
#   - 앞서 구성한 “여러 전문가가 단계별로 토론하며 답을 도출하는”
#     Zero-Shot Tree-of-Thought 프롬프트를 모델에 전달합니다.
#
#   - 모델은:
#       1) 전문가 3명의 단계별 추론을 생성하고
#       2) 의견을 비교하거나 수정하며
#       3) 마지막에 결론(정답)을 제시하는 방식으로 응답을 생성합니다.
#
# outputs[0]["generated_text"]:
#   - 모델이 생성한 전체 출력 중 텍스트 부분만 가져옵니다.
outputs = pipe(zeroshot_tot_prompt)
print(outputs[0]["generated_text"])

 Expert 1:
Step 1: Start with the initial number of apples, which is 23.

Expert 2:
Step 1: Subtract the number of apples used for lunch, which is 20.
Step 2: Add the number of apples bought, which is 6.

Expert 3:
Step 1: Start with the initial number of apples, which is 23.
Step 2: Subtract the number of apples used for lunch, which is 20.
Step 3: Add the number of apples bought, which is 6.

Results:
All three experts arrived at the same answer:

Expert 1: 23 - 20 + 6 = 9 apples
Expert 2: (23 - 20) + 6 = 9 apples
Expert 3: (23 - 20) + 6 = 9 apples

All three experts agree that the cafeteria has 9 apples left.


In [45]:
# --------------------------------------------------------------
# Zero-Shot ToT (Tree-of-Thought) 프롬프트 – 한국어 버전
# --------------------------------------------------------------
# ToT(Tree of Thought)란?
#   - 한 사람이 순차적으로만 생각을 전개하는 CoT와 달리,
#     “여러 명의 전문가가 동시에 단계별로 생각을 제시하고”
#     그중 가장 합리적인 생각으로 수렴하도록 유도하는 기법입니다.
#
#   - 논리적 문제 해결, 창의적 아이디어 도출 등에 효과적입니다.
#
# 아래 프롬프트는:
#   - 3명의 전문가가 있다고 가정하고,
#   - 각 전문가는 한 번에 1단계씩 자신의 생각을 적어 공유합니다.
#   - 진행 중에 스스로 잘못된 추론이라고 판단하면 토론에서 빠져나갑니다.
#   - 마지막에는 남은 전문가들이 의견을 모아 결론을 도출합니다.
#
# 즉, 모델이 “여러 개의 생각 경로(Thought)를 생성 → 비교 → 수렴”하는
# Tree-of-Thought 구조를 스스로 시도하도록 유도하는 제로샷 프롬프트입니다.
# --------------------------------------------------------------

zeroshot_tot_prompt = [
    {
        "role": "user",
        "content": (
            "세 명의 서로 다른 전문가가 이 문제에 답한다고 가정해 봅시다. "
            "각 전문가는 자신의 생각을 한 단계씩 적고, 그 내용을 서로에게 공유합니다. "
            "그 다음 단계에서도 모두 같은 방식으로 한 단계씩 생각을 이어갑니다. "
            "만약 어떤 전문가가 진행 중에 자신의 추론이 잘못되었다고 깨달으면, "
            "그 전문가는 토론에서 빠져나갑니다. "
            "문제는 다음과 같습니다: "
            "'급식실에 사과가 23개 있었습니다. 점심을 만드는 데 20개를 사용했고, "
            "다시 6개를 샀다면, 지금 사과는 몇 개인가요?' "
            "세 전문가가 각 단계의 생각을 공유하고, 마지막에 함께 결과를 토론하여 "
            "최종 답을 내리도록 해주세요."
        )
    }
]

outputs = pipe(zeroshot_tot_prompt)
print(outputs[0]["generated_text"])

 전문가 1: 초기 상태에서 23개의 사과가 있었습니다. 점심을 만들면 20개를 사용했으므로, 남은 사과는 3개입니다. 다시 6개를 사용했으므로, 최종적으로 사과는 9개가 남습니다.

전문가 2: 초기 상태에서 23개의 사과가 있었습니다. 점심을 만들면 20개를 사용했으므로, 남은 사과는 3개입니다. 다시 6개를 사용했으므로, 최종적으로 사과는 9개가 남습니다.

전문가 3: 초기 상태에서 23개의 사과가 있었습니다. 점심을 만들면 20개를 사용했으므로, 남은 사과는 3개입니다. 다시 6개를 사용했으므로, 최종적으로 사과는 9개가 남습니다.

토론: 전문가 1, 전문가 2, 전문가 3는 모두 같은 결과를 내었습니다. 최종적으로 사과는 9개가 남습니다.


## Zero-Shot Prompting

### ✔︎ Zero-Shot Prompting의 특징

- 예시가 필요 없다

- “무엇을 해라”라고만 하면 수행

- LLM의 일반화 능력을 그대로 이용

- JSON / 함수 호출 / 표 / 설명 / 요약 등 대부분 작업 가능

- 단, 예시가 없으므로 출력 형식이 조금 변할 가능성 있음


In [46]:
# --------------------------------------------------------------
# Zero-Shot Prompting: 예시 없이 작업을 직접 요청하기
# --------------------------------------------------------------
# zeroshot_prompt:
#   - "RPG 게임 캐릭터 프로필을 JSON 형식으로 만들어 달라"는
#     단 하나의 요청만 제공합니다.
#
#   - 예시(example)나 샘플 출력 없이도
#     모델이 지시를 이해하고 바로 결과를 생성하도록 하는 방식이 Zero-Shot Prompting입니다.
#
#   - LLM은 JSON 요구사항을 읽고:
#       { "name": ..., "class": ..., "skills": [...], ... }
#     같은 구조를 스스로 만들어냅니다.
zeroshot_prompt = [
    {"role": "user", "content": "Create a character profile for an RPG game in JSON format."}
]

# --------------------------------------------------------------
# 모델 실행: Zero-Shot 요청에 대한 JSON 캐릭터 프로필 생성
# --------------------------------------------------------------
outputs = pipe(zeroshot_prompt)

# 결과 중 생성된 텍스트만 출력
print(outputs[0]["generated_text"])

 ```json
{
  "name": "Eldrin the Wise",
  "race": "Elf",
  "class": "Wizard",
  "level": 10,
  "alignment": "Chaotic Good",
  "strength": 8,
  "dexterity": 14,
  "constitution": 12,
  "intelligence": 18,
  "wisdom": 16,
  "charisma": 10,
  "weapon_skill": "Magic",
  "armor_skill": "Light",
  "spell_slots": {
    "cantrips": ["Mage Hand", "Detect Magic", "Mage Armor", "Prestidigitation", "Identify", "Invisibility"],
    "1st level": ["Fireball", "Magic Missile", "Shield", "Cure Wounds", "Detect Thoughts", "Charm Person"],
    "2nd level": ["Light", "Hold Person", "Sleep", "Committee", "Enlarge Person", "Teleport"],
    "3rd level": ["Frostbite", "Fog Cloud", "Disintegrate", "Dimension Door", "Mirror Image", "Misty Step"]
  },
  "equipment": {
    "weapon": "Staff of the Ancients",
    "armor": "Leather Armor",
    "accessories": ["Staff of Power", "Ring of Protection", "Boots of Speed"]
  },
  "background": "Adept",
  "personality": "Curious and inventive, Eldrin is always seeking new k

In [47]:
# --------------------------------------------------------------
# Zero-Shot Prompting: 예시 없이 작업을 직접 요청하기 (한국어 버전)
# --------------------------------------------------------------
# zeroshot_prompt:
#   - "RPG 게임 캐릭터 프로필을 JSON 형식으로 만들어 달라"는
#     단 하나의 사용자 요청만 제공합니다.
#
#   - 예시(example)나 샘플 출력 없이도
#     모델이 지시 내용을 이해하고 바로 JSON 구조를 생성하는 것이
#     Zero-Shot Prompting의 핵심입니다.
#
#   - LLM은 JSON 요구사항을 읽고 예를 들어 다음과 같은 구조를 스스로 만들어냅니다:
#       { "name": ..., "class": ..., "skills": [...], ... }
zeroshot_prompt = [
    {"role": "user", "content": "RPG 게임 캐릭터 프로필을 JSON 형식으로 만들어 주세요."}
]

# --------------------------------------------------------------
# 모델 실행: Zero-Shot 요청에 대한 JSON 캐릭터 프로필 생성
# --------------------------------------------------------------
outputs = pipe(zeroshot_prompt)

# 생성된 텍스트만 출력
print(outputs[0]["generated_text"])

 ```json

{

  "name": "Aria",

  "class": "Mage",

  "level": 1,

  "stats": {

    "strength": 10,

    "dexterity": 15,

    "intelligence": 20,

    "wisdom": 12,

    "charisma": 14

  },

  "equipment": {

    "weapon": "Staff of the Ancients",

    "armor": "Leather Tunic",

    "accessories": ["Crystal Amulet", "Ring of Protection"]

  },

  "skills": {

    "spellcasting": 18,

    "alchemy": 12,

    "intimidation": 10

  },

  "background": "Aria was born into a family of scholars and mages. She has always been fascinated by the arcane arts and has dedicated her life to mastering them."

}

```


## One-shot Prompting
### ✔︎ 이 프롬프트가 좋은 이유

| 요소                            | 설명                    |
| ----------------------------- | --------------------- |
| **구조를 직접 제시**                 | 모델이 JSON 구조를 벗어나지 않음  |
| **설명 + 예시 제공**                | 출력 품질 안정화             |
| **One-shot 학습**               | 단 하나의 예시만으로도 강한 포맷 제어 |
| **Zero-shot보다 더 정확한 JSON 생성** | 형식 오류가 줄어듦            |


In [48]:
# --------------------------------------------------------------
# One-shot Learning: 출력 구조(Format)를 예시로 직접 제공
# --------------------------------------------------------------
# one_shot_template:
#   - RPG 게임 캐릭터 프로필을 생성하되,
#     반드시 JSON 형태를 따라야 한다고 명확히 지시합니다.
#
#   - 특히 예시 JSON 구조를 직접 보여주기 때문에,
#     모델은 “이 구조를 그대로 따라 작성해야 한다”고 자연스럽게 배우게 됩니다.
#
#   - 이것이 바로 One-shot Prompting:
#       ➜ 예시(example)를 하나 제공하여
#         모델의 출력 형식을 강하게 고정시키는 기법입니다.
one_shot_template = """Create a short character profile for an RPG game. Make sure to only use this format:

{
  "description": "A SHORT DESCRIPTION",
  "name": "THE CHARACTER'S NAME",
  "armor": "ONE PIECE OF ARMOR",
  "weapon": "ONE OR MORE WEAPONS"
}
"""

# 사용자 메시지로 포맷 예시를 그대로 전달
one_shot_prompt = [
    {"role": "user", "content": one_shot_template}
]

# --------------------------------------------------------------
# 모델 실행: 예시 JSON 구조를 기반으로 캐릭터 프로필 생성
# --------------------------------------------------------------
outputs = pipe(one_shot_prompt)

# 모델이 만든 JSON 형식 캐릭터 프로필 출력
print(outputs[0]["generated_text"])

 {
  "description": "A cunning rogue with a mysterious past, skilled in stealth and deception.",
  "name": "Shadowcloak",
  "armor": "Leather Hood",
  "weapon": "Dagger"
}


In [51]:
# --------------------------------------------------------------
# One-shot Learning: 출력 구조(Format)를 예시로 직접 제공 (한국어 버전)
# --------------------------------------------------------------
# one_shot_template:
#   - RPG 게임 캐릭터 프로필을 생성하되,
#     반드시 JSON 형식을 따라야 한다고 명확히 지시합니다.
#
#   - 특히 예시 JSON 구조를 직접 보여주기 때문에
#     모델은 “이 구조를 그대로 따라 작성해야 한다”는 패턴을 자연스럽게 학습합니다.
#
#   - 이것이 바로 One-shot Prompting:
#       ➜ 예시(example)를 하나 제공하여
#         모델의 출력 형식과 구조를 강하게 고정하는 기법입니다.
one_shot_template = """RPG 게임 캐릭터의 간단한 프로필을 만들어 주세요.
반드시 아래의 JSON 형식을 그대로 사용해야 합니다:

{
  "description": "간단한 설명",
  "name": "캐릭터 이름",
  "armor": "착용 중인 방어구 하나",
  "weapon": "사용하는 무기 하나 이상"
}
"""

# 사용자 메시지로 JSON 포맷 예시를 그대로 전달
one_shot_prompt = [
    {"role": "user", "content": one_shot_template}
]

# --------------------------------------------------------------
# 모델 실행: 예시 JSON 구조를 기반으로 캐릭터 프로필 생성
# --------------------------------------------------------------
outputs = pipe(one_shot_prompt)

# 모델이 생성한 JSON 캐릭터 프로필 출력
print(outputs[0]["generated_text"])

 ```json

{

  "description": "저는 애완 저녁에 찾아왔습니다. 저는 저녁 식사를 위해 먹고 있습니다.",

  "name": "애완 저녁",

  "armor": "착용 중인 저녁 반지",

  "weapon": "저녁 맛집의 치킨입니다."

}

```


### ✔︎ 아래 코드가 필요한 이유

- Colab, Jupyter, 또는 로컬 GPU 환경에서:

- LLM 모델(예: Phi-3)은 수 GB의 GPU 메모리를 사용합니다.

- del만 한다고 GPU 메모리가 완전히 비워지지 않습니다.

따라서 아래 3단계를 함께 실행해야 메모리가 정상적으로 해제됩니다:

1. del로 참조 제거

2. gc.collect()로 Python 객체 정리

3. torch.cuda.empty_cache()로 GPU 캐시 정리

In [52]:
import gc
import torch

# --------------------------------------------------------------
# 메모리 정리 단계
# --------------------------------------------------------------
# 1) del model, tokenizer, pipe
#    - 메모리를 많이 차지하는 LLM 모델 객체와 토크나이저, 파이프라인을
#      Python 변수에서 삭제합니다.
#    - 단순히 del만 하면 변수 참조가 제거되는 것이고,
#      실제 메모리 해제가 즉시 일어날 수도 있고 아닐 수도 있습니다.
del model, tokenizer, pipe

# --------------------------------------------------------------
# 2) gc.collect()
#    - Python의 가비지 컬렉터를 강제로 실행하여
#      참조가 사라진 객체들의 메모리를 회수하도록 요청합니다.
#    - 모델처럼 큰 텐서나 구조물 때문에 메모리 누수 방지하는 데 유용합니다.
gc.collect()

# --------------------------------------------------------------
# 3) torch.cuda.empty_cache()
#    - CUDA(GPU) 캐시에 남아 있는 텐서/그래프를 비워 GPU 메모리를 정리합니다.
#    - 모델을 다시 로드하거나 다른 모델로 바꿀 때 OOM(메모리 부족)을 방지할 수 있습니다.
torch.cuda.empty_cache()

# ✅ 1. llama.cpp란 무엇인가?

llama.cpp는 LLM(대형언어모델)을 ‘가볍고 빠르게’ 실행하기 위한 C++ 기반 엔진입니다.

주요 목표는:

- 저사양 컴퓨터에서도 LLM 실행

- GPU가 없어도 CPU만으로 추론 가능

- 메모리 사용을 최소화

- 속도는 최대한 빠르게

라고 이해하면 됩니다.

### ✔ 왜 llama.cpp가 중요한가?

기존 LLM 실행 방식은:

- GPU가 있어야 하고

- VRAM이 많이 필요하며

- 설치도 복잡

이런 제약들이 있습니다.

→ llama.cpp는 이를 완전히 해결해줍니다.

| 기존 방식              | llama.cpp          |
| ------------------ | ------------------ |
| GPU 필수             | CPU만으로도 가능         |
| 수 GB~수십 GB VRAM 필요 | 수 GB 미만의 RAM으로도 가능 |
| PyTorch 의존         | 가벼운 순수 C/C++ 코드    |
| 설치 복잡              | 바이너리 1개면 끝         |


# ✅ 2. GGUF란 무엇인가?

GGUF는 llama.cpp에서 사용하기 위해 ‘최적화된 LLM 파일 형식’입니다.

→ 쉽게 말해 LLM을 아주 효율적으로 저장한 파일(.gguf)


### ✔ 왜 새로운 형식(GGUF)이 필요한가?

기존 모델은 다음 특징을 가집니다:

- 크기 너무 큼 (10GB~70GB)

- 로드 시간이 오래 걸림

- CPU 실행이 어려움

그래서 llama.cpp에서 모델을 다룰 수 있도록
경량화·최적화된 압축 형식이 필요합니다.

→ 그 결과 나온 것이 GGUF입니다.

| 항목      | GGUF의 장점                               |
| ------- | -------------------------------------- |
| 파일 크기   | 기존 모델 대비 매우 작음                         |
| 속도      | CPU에서도 빠르게 추론                          |
| 호환성     | llama.cpp, LM Studio, Ollama 등에서 공통 지원 |
| 다양한 양자화 | q4, q8, fp16 등 메모리 상황에 맞춰 사용 가능        |
| 로컬 환경   | 인터넷 없이도 완전 오프라인 실행 가능                  |


## ✅ GGUF + llama.cpp 조합이 좋은 이유

두 기술이 만나면:

- 초경량 LLM 실행

- GPU 없어도 자연스러운 텍스트 생성

- 노트북에서도 GPT-4 Mini 수준 모델 사용

같은 이점을 제공합니다.

예:
- Phi-3-mini-4k-instruct-gguf
→ CPU에서도 빠르게 실행 가능한 Phi-3 모델

### llama.cpp = “컴퓨터가 약해도 LLM을 빠르게 돌려주는 엔진”

### GGUF = “이 엔진에 최적화된 초경량 모델 파일 형식”

이전 예제에서는 Transformers 기반 모델을 GPU에서 실행했지만,

이번 코드는 llama.cpp / GGUF 버전을 GPU + CPU 혼합 모드로 실행하는 방식입니다.

장점:

- 가볍고 빠르다

- CPU에서도 실행 가능

- 로컬 환경에서 추론 비용이 낮다

- Python 환경 없이도 C++ 기반 실행 가능

In [53]:
# 약 4분 소요
%pip install llama_cpp_python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 18.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.8 MB/s eta 0:00:00
  Created wheel for llama_cpp_python: filename=llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl size=4422311 sha256=26489990a989630d3ecee9dd4897ec80a555ee4d3626292a5ff076d9f4830d37
  Stored in directory: /root/.cache/pip/wheels/90/82/ab/8784ee3fb99ddb07fd36a679ddbe63122cc07718f6c1eb3be8
Successfully built llama_cpp_python


In [54]:
from llama_cpp.llama import Llama

# --------------------------------------------------------------
# llama.cpp 버전의 Phi-3 모델(GGUF)을 로드하는 코드
# --------------------------------------------------------------
# Llama.from_pretrained():
#   - Hugging Face Hub에서 GGUF 파일을 내려받아
#     llama.cpp로 실행 가능한 형태로 메모리에 로드합니다.
#
# repo_id:
#   - GGUF 모델이 저장된 Hugging Face 저장소 이름
#   - 여기서는 microsoft/Phi-3-mini-4k-instruct-gguf 사용
#
# filename:
#   - 다운로드할 GGUF 파일 이름 패턴.
#   - "*fp16.gguf" → fp16 정밀도의 gguf 파일을 선택해 가져오라는 의미
#
# n_gpu_layers:
#   - 어느 정도의 레이어를 GPU 오프로딩할지 결정
#   - -1이면 "가능한 모든 레이어를 GPU에 올림" → 최대 속도
#
# n_ctx:
#   - 컨텍스트 길이(최대 토큰 수). Phi-3 mini 4k는 4096이 최대
#
# verbose=False:
#   - 모델 로딩 과정의 상세 로그 출력 여부
llm = Llama.from_pretrained(
    repo_id="microsoft/Phi-3-mini-4k-instruct-gguf",
    filename="*fp16.gguf",
    n_gpu_layers=-1,
    n_ctx=4096,
    verbose=False
)

| 요소                          | 역할                            |
| --------------------------- | ----------------------------- |
| llama_cpp Llama API         | GGUF 모델 사용용 ChatCompletion 엔진 |
| messages                    | ChatML 구조 입력                  |
| response_format=json_object | JSON-only 출력 강제               |
| temperature=0               | 일관성 + 안정성 + 포맷 준수 강화          |
| ['choices'][0]              | 첫 번째 답변 선택                    |
| ["content"]                 | 최종 생성된 JSON 텍스트               |


In [ ]:
# --------------------------------------------------------------
# llama.cpp 기반 모델로 Chat Completion 생성 (약 7분 소요)
# --------------------------------------------------------------

output = llm.create_chat_completion(
    messages=[
        # 사용자 요청:
        #   - RPG 게임에서 사용할 '전사(Warrior)' 캐릭터를
        #     JSON 형식으로 만들어 달라는 지시.
        {"role": "user", "content": "Create a warrior for an RPG in JSON format."},
    ],

    # response_format={"type": "json_object"} :
    #   - 모델에게 "반드시 JSON 객체 형태로만 출력하라"고 강하게 지시하는 옵션.
    #   - 즉, 문자열/설명/부가 텍스트 없이 `{ ... }` JSON만 출력하도록 유도함.
    response_format={"type": "json_object"},

    # temperature=0 :
    #   - 창의성을 완전히 제거하고, 최대한 정해진 형식에 맞춰
    #     안정적이고 일관된 JSON을 생성하게 만듦.
    temperature=0,
)['choices'][0]['message']["content"]

In [ ]:
import json

# --------------------------------------------------------------
# 모델이 출력한 JSON 문자열을 실제 JSON 객체로 변환 후,
# 사람이 읽기 좋은 형태로 다시 출력하는 단계
# --------------------------------------------------------------

# json.loads(output)
#   - 모델이 생성한 `"{"name": "Thorgar", ...}"` 형식의 문자열을
#     실제 Python 딕셔너리(JSON 객체)로 변환합니다.
#
# json.dumps(..., indent=4)
#   - 변환된 JSON 객체를 다시 문자열로 출력하되,
#     indent=4 옵션으로 들여쓰기하여 예쁘게 포맷팅합니다.
#
# 즉, loads → 객체 변환, dumps → 보기 좋게 문자열로 출력
json_output = json.dumps(json.loads(output), indent=4)

# 최종 출력
print(json_output)

{
    "warrior": {
        "name": "Eldric Stormbringer",
        "class": "Warrior",
        "level": 5,
        "attributes": {
            "strength": 18,
            "dexterity": 10,
            "constitution": 16,
            "intelligence": 8,
            "wisdom": 10,
            "charisma": 12
        },
        "skills": [
            {
                "name": "Martial Arts",
                "proficiency": 20,
                "description": "Expert in hand-to-hand combat and weapon handling."
            },
            {
                "name": "Shield Block",
                "proficiency": 18,
                "description": "Highly skilled at deflecting attacks with a shield."
            },
            {
                "name": "Heavy Armor",
                "proficiency": 16,
                "description": "Expertly equipped with heavy armor for protection."
            },
            {
                "name": "Survival",
                "proficiency": 14,
                "